Deep One-Class Classification
http://proceedings.mlr.press/v80/ruff18a/ruff18a.pdf

Deep Semi-Supervised Anoamly Detection
https://arxiv.org/pdf/1906.02694.pdf


(Supervised) Contrastive Loss

CSI: Novelty Detection via Contrastive Learningon Distributionally Shifted Instances : https://arxiv.org/pdf/2007.08176.pdf

A Unifying Review of Deep and Shallow AnomalyDetection : https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9347460&tag=1

Dropout techniques on RNNs

https://adriangcoder.medium.com/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b


In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import h5py
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas as pd
from tqdm.autonotebook import tqdm
from openTSNE import TSNE
from sklearn.cluster import KMeans
from math import ceil
from glob import glob
from copy import deepcopy
from collections import deque
from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.insert(0, '../code/')
sys.path.insert(0, './models/')
import util
from models import *
import dataset
import os
import pickle
import re
import math
from glob import glob
from pathlib import Path
from sklearn.metrics import *

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

t2n = lambda x : x.cpu().detach().numpy()

parameters = {
    'input_dims' : [0,1,2,3,4,5],
    # 'input_dims' : [0,2,4],
    'num_epochs' : 128,
    'learning_rate' : 0.001,
    'weight_decay' : 10e-4,
    'eta_anom' : 1.0,
    'eta_label' : 5.0,
    'eta_network' : 1/2,
    'n_batches' : 32,
    'test_batches': 3,
    'num_hidden' : 256,
    'num_hidden_layers' : 1}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#
#
# --- NETWORK & OPTIM ---
#


# regular
model = QuenchDetectionNetworkLSTM(parameters['num_hidden'],
                                   parameters['num_hidden_layers'],
                                   parameters['eta_anom'], 
                                   parameters['eta_label'], 
                                   parameters['eta_network'],
                                   len(parameters['input_dims'])).to(device)
# folder = '../models/LSTM_256_LSTM2_Linear1/eta_anom_1_eta_label_5/'
# folder = '../models/LSTM_256_LSTM2_Linear1/amplitude_eta_anom_1_eta_label_5/'
# folder = '../models/LSTM_256_LSTM2_Linear1/only_healthly_eta_anom_1_eta_label_5/'
# folder = '../models/LSTM_256_LSTM2_Linear1/eta_anom_1_eta_label_5_32batches/'
# folder = '../models/LSTM_256_LSTM1_Linear1/eta_anom_1_eta_label_5_jan22/'

#
# from IPAC, but enable biases! 
#
# print('from IPAC, but enable biases!')
folder = '../models/LSTM_256_LSTM1_Linear1/eta_anom_1_eta_label_5_feb22_2/'  
# model.c = model.c.to(device)
# folder = '../models/LSTM_256_LSTM1_Linear1/eta_anom_1_eta_label_5_feb22_nobias_const_c/'

# linear beteween
'''
model = QuenchDetectionNetworkLSTM2LinearBetween(num_hidden = parameters['num_hidden'],
                                   num_hidden_layers = parameters['num_hidden_layers'],
                                   eta_anom = parameters['eta_anom'], 
                                   eta_label = parameters['eta_label'], 
                                   eta_network = parameters['eta_network']).to(device)
folder = '../models/LSTM_256_LSTM2_Linear1_LinearBetween/eta_anom_1_eta_label_5/'
'''

# layer norm.  
'''
model = QuenchDetectionNetworkLSTM2LayerNorm(num_hidden = parameters['num_hidden'],
                                   num_hidden_layers = parameters['num_hidden_layers'],
                                   eta_anom = parameters['eta_anom'], 
                                   eta_label = parameters['eta_label'], 
                                   eta_network = parameters['eta_network']).to(device)
folder = '../models/LSTM_256_LSTM2_Linear1_LayerNorm/eta_anom_1_eta_label_5/'
'''
'''
# softmax loss - i.e. a regular classifier
model = QuenchDetectionNetworkClassifierLSTM(num_hidden = parameters['num_hidden'],
                                   num_hidden_layers = parameters['num_hidden_layers'],
                                   eta_anom = parameters['eta_anom'], 
                                   eta_label = parameters['eta_label'], 
                                   eta_network = parameters['eta_network']).to(device)
folder = '../models/LSTM_256_LSTM1_Linear1_Classifier/eta_anom_1_eta_label_5_2(same as OC)_2/'
'''
if not os.path.exists(folder):
    print(f'creating {folder}')
    os.makedirs(folder)

optimizer = torch.optim.Adam(model.parameters(),lr=parameters['learning_rate'], weight_decay = parameters['weight_decay'])
print(model)
last_epoch = 0

weights_file = sorted(glob(folder + 'anomaly_epoch_0*'))[-1]
print(f'last epoch {last_epoch} from file {weights_file}')
last_epoch = int(weights_file.split('_')[-1])
model.load_state_dict(torch.load(weights_file,map_location = device))

In [ ]:
ae_labels = {0 : 'Quench', 
             4 : 'Jumps in probe', 
             5 : 'Time misalignment', 
             6 : 'Ringing', 
             7 : 'Pulse Cut', 
             8 :'HW C7M1A14 --> HW has been fixed now',
             10 : 'Other Anomaly',
             11 : 'Beam'}

df = pd.concat((pd.read_csv('/home/sulcan/data/quench/testRun2020_results.csv'),
                pd.read_csv('/home/sulcan/data/quench/testRun2021_results.csv'),
                pd.read_csv('/home/sulcan/data/quench/testRun2021_results_classify.csv')))

df['file'] = [file.split('/')[-1] for file in df['file']]

def get_labels(df, file, location,var):
    df_ = df[np.bitwise_and(df['file'] == file,df['location'] == location)][var]
    df_ = df_[~pd.isna(df_)]
    return list(df_.unique())

# Loading Traning Data

In [ ]:
faulty_folders = ['/home/sulcan/data/quench/down_labelled/']
faulty_files = ['/home/sulcan/data/quench/X_down_labelled_all.pickle']


healthly_files =   ['/home/sulcan/data/quench/X_up_2021-08_all.pickle',
                    '/home/sulcan/data/quench/X_up_2021-10_all.pickle', 
                    '/home/sulcan/data/quench/X_up_2021-11_all.pickle',
                    '/home/sulcan/data/quench/X_up_2022-01_all.pickle',
                     '/home/sulcan/data/quench/X_up_2022-02_all.pickle'
                   ]

healthly_folders = ['/home/sulcan/data/quench/up_filtered/2021-08-*',
                    '/home/sulcan/data/quench/up_filtered/2021-10-*',
                    '/home/sulcan/data/quench/up_filtered/2021-11-*',
                    '/home/sulcan/data/quench/up_jan/X_up_2022-01-*',
                    '/home/sulcan/data/quench/up_feb/X_up_2022-02-*',
                   ]

X_faulty, files_and_locations_faulty, pids_faulty = dataset.load_multiple_data_from_numpy(faulty_files, faulty_folders, parameters['input_dims'])
X_healthly, files_and_locations_healthly, pids_healthly = dataset.load_multiple_data_from_numpy(healthly_files, healthly_folders, parameters['input_dims'])

In [ ]:
# --- putting everything together into an input----
input_raw = X_healthly + X_faulty
files_and_locations = files_and_locations_healthly + files_and_locations_faulty
pids = pids_healthly + pids_faulty
# (A,P) -> (I,Q)
if len(parameters['input_dims']) == 6: 
    input = dataset.transform_RF_pulses(deepcopy(input_raw))
# just selecting subset of values
elif len(parameters['input_dims']) == 3:
    input = dataset.select_dimensions(deepcopy(input_raw),parameters['input_dims'])
input = dataset.normalize_data(input)

# --- creating labels ----
labels = [+1] * len(X_healthly) + [-1] * len(X_faulty)

print(f'healthly {len(X_healthly)} faulty {len(X_faulty)}')

# del X_faulty
# del X_healthly

In [ ]:
# permutation for batch indices
if os.path.isfile(folder + '/permutation.npy'):
    print('loading permutation')
    permutation = np.load(folder + '/permutation.npy', allow_pickle = True)

# Evaluation

In [ ]:
if True:
    # file = '/home/sulcan/data/quench/X_down_labelled_classified_all.pickle' # 
    # file = '/home/sulcan/data/quench/X_down_labelled_all.pickle' # 
    # file = '/home/sulcan/data/quench/X_down_all.pickle' # 
    file = '/home/sulcan/data/quench/X_up_2021-08_all.pickle'
    # file = '/home/sulcan/data/quench/X_up_2021-10_all.pickle'
    # file = '/home/sulcan/data/quench/X_up_2021-11_all.pickle'
    # file = '/home/sulcan/data/quench/X_up_2022-01_all.pickle'
    # file = '/home/sulcan/data/quench/X_up_2022-02_all.pickle'
    # file = '/home/sulcan/data/quench/X_up_2022-03_all.pickle'
    # file = '/home/sulcan/data/quench/X_up_2022-04_all.pickle'
    # file = '/home/sulcan/data/quench/X_up_2022-mai_cryo_all_03am.pickle'
    # file = '/home/sulcan/data/quench/X_up_2022-mai_cryo_all_114726.pickle'
    

    input_raw_, files_and_locations_, pids_ = dataset.load_data_from_numpy([], file, parameters['input_dims'])

    crit = lambda x : x.shape[-1] > 50
    keep = np.where(np.array(list(map(crit, input_raw_))))[0]
    input_raw_ = [input_raw_[i] for i in keep]
    files_and_locations_ = [files_and_locations_[i] for i in keep]

    skip = 1
    input_ = deepcopy(input_raw_[::skip])
    input_raw_ = input_raw_[::skip]

    # (A,P) -> (I,Q)
    if len(parameters['input_dims']) == 6: 
        input_ = dataset.transform_RF_pulses(input_)
    # just selecting subset of values
    elif len(parameters['input_dims']) == 3:
        input_ = dataset.select_dimensions(input_,parameters['input_dims'])
    input_ = dataset.normalize_data(input_)
else:
    # copying data from a permutation[0] in training set
    indices = permutation[0] + permutation[1] + permutation[2]
    input_ = [input[i] for i in indices]
    input_raw_ = [input_raw[i] for i in indices]
    files_and_locations_ = [files_and_locations [i] for i in indices]
    pids_ = [pids[i] for i in indices]
    L = [labels[i] for i in indices]
    N = len(input_)

In [ ]:
model.eval()
with torch.no_grad():
    y = [model(torch.cat([input_[i]] * 1).to(device)) for i in tqdm(range(len(input_)))]
    # y = torch.cat([y[i][1][-1] for i in range(len(y))])
    # phi = [model.phi(torch.cat([input_[i]] * 1).to(device)) for i in tqdm(range(len(input_)))]
    # phi = torch.cat([phi_[-1:,0,...] for phi_ in phi],0)
model.train()
y_ = torch.cat([y[i][1][-1] for i in range(len(y))])

### Evaluate test data (excluded from training)

In [ ]:
plt.rcParams['figure.figsize'] = [6,2]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.hist(t2n(y_).ravel(),64, color='green')
plt.ylabel('Frequency')
plt.xticks(fontsize = 14)
# plt.savefig('/home/sulcan/TUPOPT062_10.pdf')

In [ ]:
T = (y_.mean() + 2 * y_.std()).item()
sum(y_ < 0)

In [ ]:
y_ = torch.cat([y[i][1][-1] for i in range(len(y))])

plt.rcParams['figure.figsize'] = [20, 6]


plt.plot(t2n(y_),linewidth = 0.5)
plt.plot(L,'--',linewidth = 0.5)
plt.legend(['Anomaly Score of Last Pulse','Label (-1 : F, 1 H)'])
plt.show()

T = (y_.mean() + 2 * y_.std()).item()
#T = 0.0 # 0.6704126596450806
# CONFUSIO MATRIX
C = pd.DataFrame(confusion_matrix(list(map(lambda l : 0 if l == 1 else 1,L)), t2n(y_) > T), index = ['Actual H','Actual F'], columns = ['Pred H','Pred F'])
print(C)

# ROC
fpr, tpr, _ = roc_curve(list(map(lambda l : 0 if l == 1 else 1,L)), t2n(y_))
auc_score = auc(fpr.astype(np.float32),tpr.astype(np.float32))
plt.subplot(121)
plt.plot(fpr,tpr)
plt.title(f'ROC Curve (AUC {auc_score})')
plt.xlabel('False Positive Rate')
plt.ylabel('True Postitive Rate')

print(sum(y_ <= 0))

In [ ]:
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower
plt.rcParams['text.usetex'] = True
N = len(y)
T = 0.0

# order = np.random.permutation(len(y))
# order = np.where(np.array(L) == -1)[0]
# N = np.min([len(order), N])

y_ = torch.stack([y[i][1][-1,0,0] for i in range(len(y))]) # last 
order = np.argsort(t2n(y_).ravel())[::1]
order = range(len(y_))
# order = [i[0] for i in sorted(enumerate(locations), key=lambda x:x[1])]
# order = order[200:]


'''
plt.rcParams['figure.figsize'] = [6,2]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.hist(t2n(y_).ravel(),64, color='red')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.xticks(fontsize = 16)
plt.show()
# plt.savefig('/home/sulcan/hist_LAB.pdf')
'''

plt.rcParams['figure.figsize'] = [12, 2]

for i in tqdm(range(N)):
    i = order[i]

    x = input_raw_[i]
    # x = input_[i].reshape((-1,6,182)).permute((1,2,0))
    s = y[i][1].cpu().ravel()
    filename = files_and_locations_[i][0].split('/')[-1]
    location = files_and_locations_[i][1]
    
    
    # getting "GT" labels
    gt_anomaly = get_labels(df,filename, location, 'anomaly')
    gt_classify_ae = get_labels(df,filename, location, 'classify_ae')
    
    '''
    if gt_classify_ae[0] != 0:
        continue
    '''
    '''
    if len(gt_classify_ae) == 0:
        continue
    '''
    gt_classify_ae = (ae_labels[gt_classify_ae[0]] if len(gt_classify_ae) > 0 else 'NA')
    
    # ampltidue
    plt.subplot(1,2,(1))
    plt.plot(x[0,:,:],'r-')
    plt.plot(x[2,:,:],'g-')
    plt.plot(x[4,:,:],'b-')
    plt.xlabel('Time [$\mu$s]')
    plt.ylabel('Field amplitude [MV/m]')
    plt.xticks(range(0,x.shape[1],25),10 * np.arange(0,x.shape[1],25))    

    '''
    # phase
    plt.subplot(3,2,2)
    plt.plot(x[1,:,:],'r-')
    plt.plot(x[3,:,:],'g-')
    plt.plot(x[5,:,:],'b-')
    plt.title(f'anomaly(label) {gt_anomaly}\nclassify ae (label) {gt_classify_ae}')
    '''
    
    plt.subplot(1,2,(2))
    plt.plot(s[10:])
    plt.xlabel('Pulse')
    plt.ylabel('Anomaly score $s$')
    plt.subplots_adjust(bottom=0.3)
    
    print('-'.join([filename, location]))
    plt.savefig('/home/sulcan/images/q/02/' + filename + '_' + location + '_' + gt_classify_ae + '.pdf')

    '''
    plt.subplot(3,2,5)
    grad = show_input_gradients_for_output(input_[i],model, device, len(parameters['input_dims']))
    plt.plot(np.abs(grad[-1,0,:]),'r-')
    plt.plot(np.abs(grad[-1,1,:]),'g-')
    plt.plot(np.abs(grad[-1,2,:]),'b-')    
    plt.title('input grad for I')
    
    plt.subplot(3,2,6)
    grad = show_input_gradients_for_output(input_[i],model,device, len(parameters['input_dims']))
    plt.plot(np.abs(grad[-1,1,:]),'r-')
    plt.plot(np.abs(grad[-1,3,:]),'g-')
    plt.plot(np.abs(grad[-1,5,:]),'b-')
    plt.title('input grad for Q')
    '''
    plt.show()

In [ ]:
import re 
locations = []

for i,fa in enumerate(files_and_locations_):
    expr = re.findall('^C*(.*)M(.*)A(.*)L(.*)$', fa[1])[0]
    expr = 'L' + str(int(float(expr[3]))).zfill(2) + 'A' + str(int(float(expr[2]))).zfill(2) + 'M' + str(int(float(expr[1]))).zfill(2) + 'C' + str(int(float(expr[0]))).zfill(2)
    locations.append(expr)


In [ ]:
plt.rcParams['figure.autolayout'] = True
order = [i[0] for i in sorted(enumerate(locations), key=lambda x:x[1])]
plt.rcParams['figure.figsize'] = [50,2]
plt.plot([locations[i] for i in order],t2n(y_[order]))
plt.plot(t2n(y_[order]))
plt.xticks(rotation=90)

plt.savefig('/home/sulcan/scores_interlock.pdf')

# Saving results to a file

In [ ]:
df_result = pd.DataFrame(columns = ['file', 'location', 'pid', 'anomaly_score_rnn_as',\
                                    'anomaly_threshold_rnn', 'model_file', 'anomaly_ae', 'classify_ae'])
T = 0.0

rows = []
for i in tqdm(range(0,N,1)):
    i = order[i]
    
    scores = t2n(y[i][1][:]).ravel()
    anomaly = scores >= T
    
    filename = files_and_locations_[i][0].split('/')[-1]
    location = files_and_locations_[i][1]
    pid = pids_[i]
          
    pid = pid if len(pid) == len(scores) else [-1] * len(scores)
    
    for j in np.where(anomaly)[0]:
    
        score = scores[j]
        
        gt_anomaly = get_labels(df,filename, location, 'anomaly')
        gt_classify_ae = get_labels(df,filename, location, 'classify_ae')
                
        rows +=  [pd.Series(data = {'file' : "/home/xfeloper/data/MGTF/XTLReport/daq/" + filename , \
               'location' : str(location), \
               'pid' : int(pid[j]),\
               'anomaly_score_rnn_as' : float(score), \
               'anomaly_threshold_rnn' : float(T),\
               'model_file' : str(weights_file),\
               'anomaly_ae' : str((gt_anomaly[0] if len(gt_anomaly) > 0 else 'NA')),\
               'classify_ae' : str((gt_classify_ae[0] if len(gt_classify_ae) > 0 else 'NA'))}, name = i)]

df_result = df_result.append(rows,ignore_index = False)
df_result.to_csv('../results/test_set.csv')

# Embedding

In [ ]:
tsne = TSNE(n_jobs = 32, verbose = True)
phit = tsne.fit(t2n(phi))
# phit = tsne.fit(t2n(torch.cat((phi_),0)))
# phit = tsne.fit(t2n(torch.cat((phif[::2,:],model.c.view((1,-1))),0)))
# phit = tsne.fit(t2n(torch.cat((phi03, phi01, phi02),0)))

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.rcParams['text.usetex'] = True

phit = np.array(phit)

if True:
    
    N1 = phi03.shape[0]
    N2 = phi01.shape[0]
    plt.plot(phit[:N1,0],phit[:N1,1],'r.',markersize = 1)
    plt.plot(phit[N1+N2:,0],phit[N1+N2:,1],'b.',markersize = 1)
    plt.plot(phit[N1:N1+N2,0],phit[N1:N1+N2,1],'g.',markersize = 1)
    
    '''
    N = phil.shape[0]
    plt.plot(phit[N:,0],phit[N:,1],'b.',markersize = 1)
    plt.plot(phit[:N,0],phit[:N,1],'r.',markersize = 1)
    '''
    # plt.title('healthly Feb 2022 (blue), Jan 2022 (red)')
else: 
    # phif.shape
    F = np.where(np.array(labels)[indices] == -1)[0]
    H = np.where(np.array(labels)[indices] == 1)[0]

    plt.plot(phit[H,0],phit[H,1],'b.')
    plt.plot(phit[F,0],phit[F,1],'r.')
    
    plt.xlabel('Embedding Dimension 1')
    plt.ylabel('Embedding Dimension 2')
    plt.title()    
plt.savefig('/home/sulcan/TSNE_MAR_JAN_FEB.pdf')

# Visualisations for paper

In [ ]:
# TSNE embedding of classified data 
N = len(input_)
L = []
for i in range(N):
    filename = files_and_locations_[i][0].split('/')[-1]
    location = files_and_locations_[i][1]
    gt_classify_ae = get_labels(df,filename, location, 'classify_ae')
    
    L.append(gt_classify_ae[0])


In [ ]:
tsne = TSNE(n_jobs = 32, verbose = True)
phit = tsne.fit(t2n(phi))

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.rcParams['text.usetex'] = False

plt.scatter(phit[:,0],phit[:,1],c = L,cmap ='gist_rainbow',s=3)
plt.show()